In [1]:
import tensorflow as tf
import numpy as np

import pandas as pd
import cv2
import os
import math
import scipy as sp
import PIL

# Tensorflow
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2
#from keras_tuner.tuners import RandomSearch

import scikitplot as skplt
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [2]:
# batch_size: 한번에 forward & Backword 하는 샘플의 수
batch_size = 20

# Training 수
epochs = 50

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001
Decay = 1e-6

img_width = 224
img_height = 224

In [3]:
# 디렉토리 경로 설정 필요
CurrentDirectory = "/Users/krc/Documents/이어드림 수업/딥러닝_컴퓨터비전(CV)_오프라인/SkinCancer/"

train_directory = CurrentDirectory + 'TRAIN/'
test_directory  = CurrentDirectory + 'TEST/'
model_directory = CurrentDirectory + 'MODEL/'
tensorboard_directory = CurrentDirectory + 'Tensorboard'

In [4]:
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last",
    validation_split=0.10) # Train / Validation

# Online-augmentation 비적용 Generator (Test용)
datagen_test = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")




# Generator의 instance 생성 (Train)
train_generator = datagen_train.flow_from_directory(
    train_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical",
    subset = "training")

valid_generator = datagen_train.flow_from_directory(
    train_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical",
    subset = "validation")

# Generator의 instance 생성 (Test)
test_generator = datagen_test.flow_from_directory(
    test_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
    class_mode= "categorical")

Found 486 images belonging to 2 classes.
Found 54 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [6]:
# 모델 Return
EfficientNetB7_model = tf.keras.applications.EfficientNetB7(include_top=False,
    weights='imagenet', input_tensor=None, input_shape=(img_width,img_height,3), pooling=None)
# 마지막 prediction layer를 위한 작업

x = GlobalAveragePooling2D()(EfficientNetB7_model.output)
# predictions = Dense(2, activation='softmax')(x)
predictions = Dense(3, activation='sigmoid')(x)

# Input ~ Output 연결해주기
DeepLearning = Model(inputs=EfficientNetB7_model.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
DeepLearning.compile(optimizer=
         tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=Decay, amsgrad=False), 
         loss='binary_crossentropy',
         metrics=['acc']
)

/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
DeepLearning.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                            

In [8]:
# Call-back 함수

# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='MODEL/{epoch:03d}-{loss:.4f}-{acc:.4f}-{val_loss:.4f}-{val_acc:.4f}.hdf5',
            monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# tensorboard
TB = TensorBoard(log_dir='logs', write_graph=True, write_images=True)

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='val_loss',factor=0.8,patience=2, verbose=1, min_lr=1e-8)

CALLBACK = [CP, TB, LR]

In [58]:
img_width, img_height

(224, 224)

In [21]:
########## Training Start
# DeepLearning.fit(
#         train_generator,
#         # 데이터가 너무 클 경우 1-epoch을 못하는 경우
#         # steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        
#         epochs=15,
#         callbacks=CALLBACK,
#         shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
#         validation_data=valid_generator)
###########

Epoch 1/15


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/model_3/dense_3/MatMul/MatMul' defined at (most recent call last):
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in

In [9]:
######### Training Start
DeepLearning.fit(train_generator, 
                 epochs=20, 
                 callbacks=CALLBACK,
                 shuffle=True, 
                 validation_data=valid_generator)

/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20


2022-07-25 09:55:44.997321: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/binary_crossentropy/logistic_loss/mul/BroadcastGradientArgs' defined at (most recent call last):
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/tornado/plat

In [53]:
train_generator.image_shape

(224, 224, 3)

In [62]:
# 성능 검사
DeepLearning.evaluate(valid_generator) # 괄호에 val GENERATOR를 넣는다. 

2/2 [==============================] - 5s 2s/step - loss: 0.7089 - acc: 0.5000


[0.7089135646820068, 0.5]

In [63]:
Predicted = DeepLearning.predict(test_generator)

# CheckSame = [Pred == Label for Pred, Label in zip(Predicted>0.5, Test_y == 1)]
# pd.Series(CheckSame).value_counts()

/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/krc/miniforge3/envs/tensorflow/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [65]:
Result = pd.DataFrame(Predicted)


In [66]:
Result

,0
0,0.411977
1,0.411938
2,0.411851
3,0.411926
4,0.411965
5,0.411943
6,0.411868
7,0.411836
8,0.411863
9,0.411873


## VGG16